# Adapter Pattern

## Intent
Convert the interface of a class into another interface clients expect. Adapter lets classes work together that couldn't otherwise because of incompatible interfaces.

## Problem
You need to use an existing class, but its interface doesn't match what you need:
- Legacy system integration
- Third-party library with different interface
- Multiple data sources with different APIs
- Incompatible components

**Real-world analogy**: Power adapter for different electrical outlets

## When to Use
✅ **Use when:**
- Want to use existing class with incompatible interface
- Need to integrate legacy code
- Want to create reusable class for unrelated classes
- Multiple implementations need uniform interface

❌ **Avoid when:**
- Can modify the original interface
- Simple wrapper is sufficient
- Performance overhead is critical

## Pattern Structure
```
┌────────┐         ┌─────────┐
│ Client │────────►│  Target │
└────────┘         │Interface│
                   └─────────┘
                        ▲
                        │
                   ┌────┴─────┐
                   │ Adapter  │
                   ├──────────┤
                   │ adaptee  │───►┌──────────┐
                   └──────────┘    │ Adaptee  │
                                   │(existing)│
                                   └──────────┘
```

## Example 1: Payment Gateways (Without Adapter)

**Problem**: Different payment APIs with different interfaces

In [ ]:
# WITHOUT Adapter - Different interfaces are confusing

# Existing PayPal library
class PayPalAPI:
    def send_payment(self, amount, email):
        print(f"PayPal: Sending ${amount} to {email}")
        return True

# Existing Stripe library  
class StripeAPI:
    def charge(self, cents, token):
        print(f"Stripe: Charging {cents} cents with token {token}")
        return {"success": True}

# Client code has to know about both!
def process_payment(gateway, amount, details):
    if isinstance(gateway, PayPalAPI):
        return gateway.send_payment(amount, details['email'])
    elif isinstance(gateway, StripeAPI):
        result = gateway.charge(int(amount * 100), details['token'])
        return result['success']
    # Adding new gateway requires modifying this function!

# Usage - messy
paypal = PayPalAPI()
stripe = StripeAPI()
process_payment(paypal, 100, {'email': 'user@example.com'})
process_payment(stripe, 100, {'token': 'tok_123'})

## Implementation: Adapter Pattern

In [ ]:
from abc import ABC, abstractmethod

# Target interface (what client expects)
class PaymentProcessor(ABC):
    """Unified payment interface."""
    
    @abstractmethod
    def process(self, amount: float, customer_id: str) -> bool:
        """Process payment."""
        pass


# Adaptees (existing incompatible interfaces)
class PayPalAPI:
    """Existing PayPal library."""
    def send_payment(self, amount: float, email: str):
        print(f"  🅿️  PayPal: Sending ${amount:.2f} to {email}")
        return True


class StripeAPI:
    """Existing Stripe library."""
    def charge(self, cents: int, token: str):
        print(f"  💳 Stripe: Charging {cents} cents with token {token}")
        return {"success": True, "transaction_id": "txn_123"}


class SquareAPI:
    """Existing Square library."""
    def make_payment(self, amount_cents: int, customer_ref: str):
        print(f"  ⬛ Square: Processing {amount_cents} cents for {customer_ref}")
        return {"status": "completed"}


# Adapters (convert adaptee interface to target interface)
class PayPalAdapter(PaymentProcessor):
    """Adapts PayPal to PaymentProcessor interface."""
    
    def __init__(self):
        self.paypal = PayPalAPI()
    
    def process(self, amount: float, customer_id: str) -> bool:
        # Adapt the interface
        email = f"{customer_id}@example.com"  # Convert ID to email
        return self.paypal.send_payment(amount, email)


class StripeAdapter(PaymentProcessor):
    """Adapts Stripe to PaymentProcessor interface."""
    
    def __init__(self):
        self.stripe = StripeAPI()
    
    def process(self, amount: float, customer_id: str) -> bool:
        # Adapt the interface
        cents = int(amount * 100)  # Convert dollars to cents
        token = f"tok_{customer_id}"  # Generate token from ID
        result = self.stripe.charge(cents, token)
        return result["success"]


class SquareAdapter(PaymentProcessor):
    """Adapts Square to PaymentProcessor interface."""
    
    def __init__(self):
        self.square = SquareAPI()
    
    def process(self, amount: float, customer_id: str) -> bool:
        # Adapt the interface
        amount_cents = int(amount * 100)
        result = self.square.make_payment(amount_cents, customer_id)
        return result["status"] == "completed"


# Client code (works with any payment processor)
def checkout(processor: PaymentProcessor, amount: float, customer_id: str):
    """Process checkout with any payment processor."""
    print(f"\n💰 Processing ${amount:.2f} for customer {customer_id}")
    success = processor.process(amount, customer_id)
    if success:
        print("  ✅ Payment successful!")
    else:
        print("  ❌ Payment failed!")
    return success


# Demo
print("=== Payment Processing with Adapters ===")

# Client doesn't know about specific APIs!
processors = [
    PayPalAdapter(),
    StripeAdapter(),
    SquareAdapter()
]

for processor in processors:
    checkout(processor, 99.99, "customer_123")

## Real-World Example: Data Source Adapters

In [ ]:
# Target interface
class DataReader(ABC):
    """Unified interface for reading data."""
    
    @abstractmethod
    def get_data(self) -> list:
        pass


# Existing incompatible data sources
class SQLDatabase:
    """Legacy SQL database."""
    def execute_query(self, query: str):
        print(f"  🗄️  SQL: Executing '{query}'")
        return [("Alice", 30), ("Bob", 25)]


class RESTApi:
    """External REST API."""
    def fetch_json(self, endpoint: str):
        print(f"  🌐 API: Fetching from '{endpoint}'")
        return {
            "users": [
                {"name": "Charlie", "age": 35},
                {"name": "Diana", "age": 28}
            ]
        }


class CSVFile:
    """CSV file reader."""
    def read_lines(self, filename: str):
        print(f"  📄 CSV: Reading '{filename}'")
        return [
            "Eve,32",
            "Frank,29"
        ]


# Adapters
class SQLAdapter(DataReader):
    """Adapts SQL database to DataReader."""
    
    def __init__(self, database: SQLDatabase):
        self.database = database
    
    def get_data(self) -> list:
        results = self.database.execute_query("SELECT name, age FROM users")
        # Convert tuples to dictionaries
        return [{"name": name, "age": age} for name, age in results]


class APIAdapter(DataReader):
    """Adapts REST API to DataReader."""
    
    def __init__(self, api: RESTApi):
        self.api = api
    
    def get_data(self) -> list:
        response = self.api.fetch_json("/users")
        # Extract users from response
        return response["users"]


class CSVAdapter(DataReader):
    """Adapts CSV file to DataReader."""
    
    def __init__(self, csv_file: CSVFile):
        self.csv_file = csv_file
    
    def get_data(self) -> list:
        lines = self.csv_file.read_lines("users.csv")
        # Parse CSV lines
        data = []
        for line in lines:
            name, age = line.split(",")
            data.append({"name": name, "age": int(age)})
        return data


# Client code
def process_users(reader: DataReader):
    """Process users from any data source."""
    users = reader.get_data()
    for user in users:
        print(f"    • {user['name']}: {user['age']} years old")


# Demo
print("\n=== Data Source Adapters ===")

print("\nFrom SQL Database:")
sql_reader = SQLAdapter(SQLDatabase())
process_users(sql_reader)

print("\nFrom REST API:")
api_reader = APIAdapter(RESTApi())
process_users(api_reader)

print("\nFrom CSV File:")
csv_reader = CSVAdapter(CSVFile())
process_users(csv_reader)

## Real-World Example: Media Player Adapter

In [ ]:
# Target interface
class MediaPlayer(ABC):
    """Unified media player interface."""
    
    @abstractmethod
    def play(self, filename: str) -> None:
        pass


# Existing incompatible players
class MP3Player:
    """Plays MP3 files only."""
    def play_mp3(self, filename: str):
        print(f"  🎵 Playing MP3: {filename}")


class WAVPlayer:
    """Plays WAV files only."""
    def load_wav(self, filename: str):
        print(f"  🎵 Loading WAV: {filename}")
    
    def start(self):
        print(f"  ▶️  Playing WAV file")


class FLACPlayer:
    """Plays FLAC files only."""
    def initialize(self, filename: str):
        print(f"  🎵 Initializing FLAC: {filename}")
    
    def decode_and_play(self):
        print(f"  ▶️  Decoding and playing FLAC")


# Adapters
class MP3Adapter(MediaPlayer):
    """Adapts MP3Player to MediaPlayer."""
    
    def __init__(self):
        self.player = MP3Player()
    
    def play(self, filename: str):
        self.player.play_mp3(filename)


class WAVAdapter(MediaPlayer):
    """Adapts WAVPlayer to MediaPlayer."""
    
    def __init__(self):
        self.player = WAVPlayer()
    
    def play(self, filename: str):
        self.player.load_wav(filename)
        self.player.start()


class FLACAdapter(MediaPlayer):
    """Adapts FLACPlayer to MediaPlayer."""
    
    def __init__(self):
        self.player = FLACPlayer()
    
    def play(self, filename: str):
        self.player.initialize(filename)
        self.player.decode_and_play()


# Universal media player
class UniversalPlayer:
    """Plays any audio format using adapters."""
    
    def __init__(self):
        self.players = {
            "mp3": MP3Adapter(),
            "wav": WAVAdapter(),
            "flac": FLACAdapter()
        }
    
    def play(self, filename: str):
        extension = filename.split(".")[-1].lower()
        
        if extension in self.players:
            print(f"\n🎼 Playing {filename}")
            self.players[extension].play(filename)
        else:
            print(f"\n❌ Unsupported format: {extension}")


# Demo
print("=== Universal Media Player ===")
player = UniversalPlayer()

player.play("song.mp3")
player.play("audio.wav")
player.play("music.flac")
player.play("video.mp4")  # Unsupported

## Class Adapter vs Object Adapter

### Object Adapter (Composition - Recommended)
```python
class Adapter(Target):
    def __init__(self):
        self.adaptee = Adaptee()  # Composition
```

### Class Adapter (Multiple Inheritance)
```python
class Adapter(Target, Adaptee):  # Multiple inheritance
    pass
```

In [ ]:
# Demonstration of both approaches

# Adaptee
class OldPrinter:
    def print_text(self, text: str):
        print(f"Old Printer: {text}")

# Target
class ModernPrinter(ABC):
    @abstractmethod
    def print(self, text: str):
        pass

# Object Adapter (Composition)
class ObjectAdapter(ModernPrinter):
    def __init__(self, old_printer: OldPrinter):
        self.old_printer = old_printer
    
    def print(self, text: str):
        self.old_printer.print_text(text)

# Class Adapter (Inheritance) - Python supports this
class ClassAdapter(ModernPrinter, OldPrinter):
    def print(self, text: str):
        self.print_text(text)

# Demo
print("\nObject Adapter:")
adapter1 = ObjectAdapter(OldPrinter())
adapter1.print("Hello via Object Adapter")

print("\nClass Adapter:")
adapter2 = ClassAdapter()
adapter2.print("Hello via Class Adapter")

## Advantages & Disadvantages

### ✅ Advantages
1. **Single Responsibility**: Separate interface conversion
2. **Open/Closed Principle**: Add adapters without modifying existing code
3. **Reuse existing code**: Work with incompatible interfaces
4. **Flexibility**: Switch implementations easily
5. **Integration**: Connect disparate systems

### ❌ Disadvantages
1. **Complexity**: More classes to manage
2. **Performance**: Extra indirection layer
3. **Maintenance**: Keep adapters synchronized with adaptees

## When to Use Adapter vs Other Patterns

**Adapter**: Make existing interfaces work together
```python
adapter = StripeAdapter()  # Adapts Stripe to PaymentProcessor
```

**Decorator**: Add behavior, keep interface same
```python
encrypted = EncryptionDecorator(file)  # Adds encryption
```

**Proxy**: Control access, same interface
```python
proxy = DatabaseProxy()  # Controls access to database
```

**Facade**: Simplify complex interface
```python
facade = SystemFacade()  # Simplifies subsystem
```

## Best Practices

1. **Prefer composition**: Use object adapter over class adapter
2. **Keep adapters thin**: Minimal logic, just translation
3. **Document mappings**: Explain how interfaces are adapted
4. **Test thoroughly**: Ensure correct behavior preservation
5. **Consider caching**: If adaptation is expensive

## Related Patterns

- **Bridge**: Separates abstraction from implementation (design time)
- **Decorator**: Adds behavior (doesn't change interface)
- **Proxy**: Controls access (keeps interface)
- **Facade**: Simplifies interface (doesn't adapt)

## Summary

Adapter pattern enables:
- Interface conversion
- Legacy system integration
- Third-party library adaptation
- Multiple implementation support

Perfect for: API integration, legacy code, third-party libraries, multiple data sources.

**Key Insight**: Make incompatible interfaces work together through translation!